In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

print(housing.feature_names)

X = pd.DataFrame(housing.data, columns=housing.feature_names)#[["AveRooms", "Latitude", "Longitude"]]

y = housing.target

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

rmse = mse ** 0.5
print(f"Root mean squared error: {rmse:.4f}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.4f}")


Root mean squared error: 0.7341
R-squared: 0.5888
